In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('wine.csv')

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,bad
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,bad
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,bad
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,good
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,bad


In [4]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [5]:
df['quality']=df['quality'].replace('bad',0)

In [6]:
df['quality']=df['quality'].replace('good',1)

In [7]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [8]:
df.shape

(1599, 12)

In [9]:
y=df['quality']

In [10]:
X=df.drop(['quality'],1)

<ipython-input-10-65df9f6ad88e>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X=df.drop(['quality'],1)


In [11]:
df['quality'].value_counts()

1    855
0    744
Name: quality, dtype: int64

### the dataset is balanced

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0,1))

In [13]:
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=1,stratify=y)

### we split and scale the data and then we test random forest classifier

In [15]:
scaler.fit(X_train)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
rfc=RandomForestClassifier(n_estimators=200)

In [17]:
rfc.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200)

In [18]:
rfc_pred=rfc.predict(X_test)

In [19]:
print(classification_report(y_test,rfc_pred))

              precision    recall  f1-score   support

           0       0.82      0.75      0.78       149
           1       0.80      0.85      0.82       171

    accuracy                           0.81       320
   macro avg       0.81      0.80      0.80       320
weighted avg       0.81      0.81      0.81       320



In [20]:
confusion_matrix(y_test,rfc_pred)

array([[112,  37],
       [ 25, 146]], dtype=int64)

### the score was 0.8, not bad not very good

In [35]:
from sklearn.model_selection import RandomizedSearchCV

In [36]:
param_grid={'n_estimators': [100,800],
 'min_samples_split': [2,3],
 'min_samples_leaf': [1,2],
 'max_features': ['auto','sqrt'],
 'max_depth': [100,110],
 'bootstrap': [True,False]}

In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [39]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 64 candidates, totalling 192 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [100, 110],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 3],
                         'n_estimators': [100, 800]},
             verbose=2)

In [40]:
rf_pred=grid_search.predict(X_test)

In [41]:
print(classification_report(y_test,rf_pred))

              precision    recall  f1-score   support

           0       0.80      0.76      0.78       149
           1       0.80      0.83      0.81       171

    accuracy                           0.80       320
   macro avg       0.80      0.79      0.80       320
weighted avg       0.80      0.80      0.80       320



In [64]:
from sklearn.linear_model import LogisticRegression

In [71]:
param_grid_lr = {
    'max_iter': [20, 50, 100, 200, 500, 1000],                      
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],   
    'class_weight': ['balanced']                                    
}

In [72]:
lr=LogisticRegression()

In [73]:
grid_search = GridSearchCV(estimator = lr, param_grid = param_grid_lr, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [74]:
grid_search.fit(X_train,y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


GridSearchCV(cv=3, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'class_weight': ['balanced'],
                         'max_iter': [20, 50, 100, 200, 500, 1000],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             verbose=2)

In [77]:
print(grid_search.best_estimator_)

LogisticRegression(class_weight='balanced', max_iter=20, solver='newton-cg')


In [75]:
lr_pred=grid_search.predict(X_test)

In [76]:
print(classification_report(y_test,lr_pred))

              precision    recall  f1-score   support

           0       0.67      0.72      0.70       149
           1       0.74      0.69      0.72       171

    accuracy                           0.71       320
   macro avg       0.71      0.71      0.71       320
weighted avg       0.71      0.71      0.71       320



In [27]:
from sklearn.ensemble import BaggingClassifier

In [39]:
from sklearn.tree import DecisionTreeClassifier

In [58]:
bc = BaggingClassifier()

In [61]:
n_estimators = [100, 300, 500, 800, 1200]
max_samples = [5, 10, 25, 50, 100]
max_features = [1, 2, 5, 10, 13]
hyperbag = dict(n_estimators = n_estimators, max_samples = max_samples, 
              max_features = max_features)
grid = GridSearchCV(bc, hyperbag, cv = 3, verbose = 1, n_jobs = -1)

In [ ]:
bc_grid = grid.fit(X_train, y_train)

In [63]:
grid.best_params_

{'max_features': 10, 'max_samples': 100, 'n_estimators': 100}

In [64]:
bc1 = BaggingClassifier(max_features=10, max_samples=100,n_estimators= 100)
bc1.fit(X_train,y_train)

BaggingClassifier(max_features=10, max_samples=100, n_estimators=100)

In [65]:
bc_pred=bc1.predict(X_test)

In [66]:
print(classification_report(y_test,bc_pred))

              precision    recall  f1-score   support

           0       0.74      0.70      0.72       149
           1       0.75      0.78      0.77       171

    accuracy                           0.75       320
   macro avg       0.75      0.74      0.74       320
weighted avg       0.75      0.75      0.75       320



In [ ]:
import xgboost as xgb

In [68]:
param={"learning_rate"    : [0.05, 0.15, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 5, 6,10,15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.1 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3,  0.5 , 0.7 ] }

In [70]:
from xgboost.sklearn import XGBClassifier

In [72]:
xgb=XGBClassifier()

In [73]:
grid = GridSearchCV(xgb, param, cv = 3, verbose = 1, n_jobs = -1)

In [ ]:
xgb_grid = grid.fit(X_train, y_train)

In [75]:
grid.best_params_

{'colsample_bytree': 0.5,
 'gamma': 0.3,
 'learning_rate': 0.15,
 'max_depth': 15,
 'min_child_weight': 3}

In [ ]:
xgb=XGBClassifier(colsample_bytree=0.5,n_estimators=500,
 gamma= 0.3,
 learning_rate= 0.15,
 max_depth= 15,
 min_child_weight= 3)
xgb.fit(X_train,y_train)

In [81]:
xgb_pred=xgb.predict(X_test)

In [82]:
print(classification_report(y_test,xgb_pred))

              precision    recall  f1-score   support

           0       0.79      0.70      0.74       149
           1       0.76      0.84      0.80       171

    accuracy                           0.78       320
   macro avg       0.78      0.77      0.77       320
weighted avg       0.78      0.78      0.77       320



In [83]:
from sklearn.svm import SVC

In [84]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

In [85]:
grid = GridSearchCV(SVC(), param_grid, refit = True, cv=3, verbose = 3)

In [88]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV 1/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.724 total time=   0.0s
[CV 2/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.728 total time=   0.0s
[CV 3/3] END ........C=0.1, gamma=1, kernel=rbf;, score=0.756 total time=   0.1s
[CV 1/3] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.534 total time=   0.0s
[CV 2/3] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.535 total time=   0.1s
[CV 3/3] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.535 total time=   0.1s
[CV 1/3] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.534 total time=   0.0s
[CV 2/3] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.535 total time=   0.1s
[CV 3/3] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.535 total time=   0.0s
[CV 1/3] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.534 total time=   0.1s
[CV 2/3] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.535 total time=   0.1s
[CV 3/3] END ....C=0.1, gamma=0.001, kernel=rbf;

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [89]:
svc= grid.predict(X_test)

In [90]:
print(classification_report(y_test,svc))

              precision    recall  f1-score   support

           0       0.74      0.77      0.76       149
           1       0.79      0.77      0.78       171

    accuracy                           0.77       320
   macro avg       0.77      0.77      0.77       320
weighted avg       0.77      0.77      0.77       320



In [91]:
grid.best_params_

{'C': 100, 'gamma': 1, 'kernel': 'rbf'}